Analizando Doja Cat Lyrics com Python

In [74]:
%matplotlib inline

import pandas as pd
import string
import seaborn as sns
import matplotlib.pyplot as plt
import collections
import nltk
from nltk.tokenize import word_tokenize
from nltk.sentiment import SentimentIntensityAnalyzer

Explorando o Dataset

In [75]:
all_lyrics = pd.read_csv('Doja_Cat_lyrics_all.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'Doja_Cat_lyrics_all.csv'

In [ ]:
all_lyrics.columns

In [ ]:
all_lyrics.info()

In [ ]:
all_lyrics.head()

In [ ]:
all_lyrics.tail()

Limpando o Dataset

In [ ]:
#Alterando Lyrics para minúscula
all_lyrics['new_lyrics'] = all_lyrics['Lyric'].str.lower()
#Removendo pontuações
all_lyrics['new_lyrics']= all_lyrics['new_lyrics'].str.replace('[^\w\s]',' ')
#Removendo Stopwords
stop = ['is','it','oh','of','uh','that','this','yeah','le','might',
        'a','am', 'was', 'were', 'been','la','hey','b', 'n','is','to','end','in','the','t']

all_lyrics['new_lyrics'] = all_lyrics['new_lyrics'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [ ]:
words_affirmation = ['love','friend','friendship','great','proud','baby','mine','feel','feelings','perfect',
                     'precious','like you','beauty']
quality_time = ['watch','never','now','time','spend','text','texting','follow','chill','alone','play',
                   'stay','party','talking','talk','sleep']
physical_touch = ['kiss','love','sex','fuck','eyes','fight','hold','sexuality','face','body','mouth','dick',
                'ass','touch','eye','pussy','eye']
acts_service = ['turn','clean','stay','act','patient','relax','drive','notice','work','need','wait','focused',
              'influence', 'struggle','sleep','share','sharing','honest']
receiving_gifts = ['weed','sweet','ticket','bussines','send','spent','t-shirt','tiffanys','camera','getting','get',
            'money','cash','more','ring','picture','clothes']

In [ ]:
#Criando uma categoria para cada lista de palavra
words_affirmation_regex = '|'.join(words_affirmation)
quality_time_regex = '|'.join(quality_time)
physical_touch_regex = '|'.join(physical_touch)
acts_service_regex = '|'.join(acts_service)
receiving_gifts_regex = '|'.join(receiving_gifts)

#Nova coluna para cada categoria
all_lyrics['Words of Affirmation'] = all_lyrics['new_lyrics'].str.contains(words_affirmation_regex)
all_lyrics['Quality Time'] = all_lyrics['new_lyrics'].str.contains(quality_time_regex)
all_lyrics['Physical Touch'] = all_lyrics['new_lyrics'].str.contains(physical_touch_regex)
all_lyrics['Acts of Service'] = all_lyrics['new_lyrics'].str.contains(acts_service_regex)
all_lyrics['Receiving Gifts'] = all_lyrics['new_lyrics'].str.contains(receiving_gifts_regex)

#Contando e realizando soma de quantidade de vezes em que as palavras da categoria são mencionadas 
words_affirmation_count = sum(all_lyrics['Words of Affirmation'])
quality_time_count = sum(all_lyrics['Quality Time'])
physical_touch_count = sum(all_lyrics['Physical Touch'])
acts_service_count = sum(all_lyrics['Acts of Service'])
receiving_gifts_count = sum(all_lyrics['Receiving Gifts'])

#print
print("Words of Affirmation: ", words_affirmation_count)
print("Quality Time: ", quality_time_count)
print("Physical Touch: ", physical_touch_count)
print("Acts of Service: ", acts_service_count)
print("Receiving Gifts: ", receiving_gifts_count)



Selecionando as duas maiores linguagens para vizualisar as menções através dos a albums

In [ ]:
#Novo dataframe agrupado pelo album
mentions = all_lyrics.groupby('Album Name').sum().reset_index()

In [ ]:
mentions.sort_values(by='Words of Affirmation', ascending=False)

In [ ]:
mentions.sort_values(by='Quality Time', ascending=False)

Entendendo o foco dos lyrics da Doja Cat através do tokenice

In [ ]:
#Criando token das palavras
all_lyrics['lyrics_tok'] = all_lyrics['new_lyrics'].apply(lambda x: word_tokenize(x))

In [ ]:
#Determinando palavras frequentemente usadas e criando nova lista.
word_list = [word for list_ in all_lyrics['lyrics_tok'] for word in list_]
#Função de contagem
word_frequency = collections.Counter(word_list)
#Organizando a frequência
word_frequency = sorted(word_frequency.items(), key=lambda x: x[1], reverse=True)
word_frequency

Entendendo sentimentos dos Lyrics

In [ ]:
#Função para análise de sentimentos
sia = SentimentIntensityAnalyzer()
#Teste
sia.polarity_scores("I like Doja Cat!")

In [ ]:
#Criando nova coluna e aplicando sia com a expressão lambda
all_lyrics['polarity'] = all_lyrics['new_lyrics'].apply(lambda x: sia.polarity_scores(x))

In [ ]:
all_lyrics[['neg', 'neu', 'pos', 'compound']] = all_lyrics['polarity'].apply(pd.Series)
#Excluindo coluna
all_lyrics.drop('polarity', axis=1)

In [ ]:
#Calculando sentimento
pos = sum(all_lyrics['pos'])
neu = sum(all_lyrics['neu'])
neg = sum(all_lyrics['neg'])
compound = sum(all_lyrics['compound'])

#Print
print("positive: ", pos)
print("neutral: ", neu)
print("negative: ", neg)
print("compound: ", compound)

Descobrindo os sentimentos referentes a cada linguagem do amor

In [ ]:
#Criando dataframes
words_affirmation = all_lyrics[all_lyrics['Words of Affirmation']==True]
quality_time = all_lyrics[all_lyrics['Quality Time']==True]
physical_touch = all_lyrics[all_lyrics['Physical Touch']==True]
acts_service = all_lyrics[all_lyrics['Acts of Service']==True]
receiving_gifts = all_lyrics[all_lyrics['Receiving Gifts']==True]

In [ ]:
#Calculando sentimentos de cada dataframe utilizando o compound
words_affirmation_sentiment = words_affirmation['compound'].sum()
quality_time_sentiment = quality_time['compound'].sum()
physical_touch_sentiment = physical_touch['compound'].sum()
acts_service_sentiment =acts_service['compound'].sum()
receiving_gifts_sentiment = receiving_gifts['compound'].sum()

#Print
print("Words of Affirmation: ", words_affirmation_sentiment)
print("Quality Time: ", quality_time_sentiment)
print("Physical Touch: ", physical_touch_sentiment)
print("Acts of Service: ", acts_service_sentiment)
print("Receiving Gifts: ", receiving_gifts_sentiment)